# Value Based Window Functions

These are neither Ranking or Aggregate functions, but instead they make use of the values within your data and let you highlight some of them individually in a way that would otherwise prove difficult.

For example, we may want to see our data offset slightly in order to perform month on month sales reports.  Or maybe we wish to highlight the first or last orders taken (these may not be the minimum or maximum, but simply the first seen).

#### FIRST_VALUE

In order to ascertain the first value we are seeing in our dataset we need to use the FIRST_VALUE Window Function.

It works as follows:

![FirstValue](https://www.sqlworld.co.uk/images/Blogs/SQLServer/FirstValue.jpg)

The visual shows that the first record we saw was the Score of 100 and that is the record returned to us.  Crucially you can see that the Score column is not ordered as this function takes the first value it sees, not specifically the minimum or maximum of a column.

To clarify this we shall look at our table sales.salesOrderHeaderWindow:

In [ ]:
select *
from JupyterDatabase.sales.salesOrderHeaderWindow
where salesPersonID = 51
order by orderDate;

We can see from the output above that our first order is 2019-01-01 and the amount is for £265.

Therefore if we wished to see our minimum orderDate we could use MIN, but what if it is the £265 that we wish to see throughout our results as the first value we sold?

This is where FIRST_VALUE can come in.  Within our OVER clause we can tell SQL Server to order the results by orderDate, but we then ask FIRST_VALUE to return our saleAmount:

In [ ]:
select first_value(saleAmount) over(order by orderDate), *
from JupyterDatabase.sales.salesOrderHeaderWindow
where salesPersonID = 51;

As we can see, this works perfectly, showing us the value of our first order throughout the query.

#### LAST_VALUE

As you can imagine, LAST_VALUE is identical to FIRST_VALUE except that it highlights the last record seen rather than that first.

![LastValue](https://www.sqlworld.co.uk/images/Blogs/SQLServer/LastValue.jpg)

During Aggregates we heavily emphasised the fact that Window Functions look backwards by default and that if we want it to look forwards in the dataset then we need to explicitly tell it to do so.

As LAST_VALUE by definition is looking for the last record seen, we need to add such a frame to our query:

In [ ]:
select last_value(saleAmount)
		over
		(
			order by orderDate
			rows between unbounded preceding and unbounded following
		), *
from JupyterDatabase.sales.salesOrderHeaderWindow
where salesPersonID = 51;

With the frame in place we get the exact output we are looking to achieve.

#### LEAD

The final pairing within Window Functions are LEAD and LAG.  We shall start with LEAD.

A common requirement of reporting is to offset data so that we can easily calculate such values as 'month on month growth' or 'year on year sales'.

These can require looking either backwards or forwards depending on your requirement and data.

The LEAD function looks forwards in time across your data as shown here:

![Lead](https://www.sqlworld.co.uk/images/Blogs/SQLServer/Lead.jpg)

Simply, it offsets your data and places it alongside your existing data.  If no offset is available (in the above picture we cannot offset row 5 as there is no row 6 present) SQL Server returns a NULL value.

The syntax is the same in regard to the OVER clause, but the LEAD portion itself is slightly different as it receives not only the column we wish to offset but also by how many steps.

For example, if we have daily data but wish to compare Sunday to Sunday, Monday to Monday etc for a more valid comparison we would pass the number 7 to the function in order to offset the daily data by 7 each time.

Let's look at an example:

In [ ]:
select *
from JupyterDatabase.sales.salesOrderHeaderWindow
where salesPersonID = 44
order by orderDate;

SalesPersonID 44 has made three sales, each on consecutive days.

We shall use LEAD in order to place each order next to the one from the following day:

In [ ]:
select *, lead(saleAmount, 1) over(order by orderDate)
from JupyterDatabase.sales.salesOrderHeaderWindow
where salesPersonID = 44
order by orderDate;

As mentioned, we can increase the offset if desired (only up to 2 in this case as we only have 3 records to start with):

In [ ]:
select *, lead(saleAmount, 2) over(order by orderDate)
from JupyterDatabase.sales.salesOrderHeaderWindow
where salesPersonID = 44
order by orderDate;

#### LAG

Moving straight into LAG we find that, in syntax, it is identical to LEAD.  The difference between the two is that LEAD looks 'forward' to the next record down whereas LAG looks 'backwards', referencing records above in the table.

![Lag](https://www.sqlworld.co.uk/images/Blogs/SQLServer/Lag.jpg)

Note, again as with LEAD, if there are no records to offset to then SQL Server returns NULL values.

We shall run the same examples as with LEAD, but this time using LAG:

In [ ]:
select *, lag(saleAmount, 1) over(order by orderDate)
from JupyterDatabase.sales.salesOrderHeaderWindow
where salesPersonID = 44
order by orderDate;

And, as before, with a larger offset:

In [ ]:
select *, lag(saleAmount, 2) over(order by orderDate)
from JupyterDatabase.sales.salesOrderHeaderWindow
where salesPersonID = 44
order by orderDate;